In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
import os
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Establezco una ruta absoluta a un directorio existente de mi Google Drive
BASE_FOLDER = "/content/drive/Othercomputers/Mi portátil/Master/TFM/TFM/Datos"

In [4]:
train_dir =BASE_FOLDER +"/Train"
test_dir = BASE_FOLDER +"/Test"
val_dir = BASE_FOLDER +"/Valid"

In [5]:
def cargar_matriz_3d_de_archivo(ruta_archivo, N):
    """
    Carga una matriz 3D de tamaño N a partir de un archivo de texto que contiene
    las coordenadas de los elementos que son 1. El resto de elementos serán 0.

    :param ruta_archivo: str - Ruta al archivo de texto.
    :param N: int - Tamaño de la matriz 3D (N x N x N).
    :return: np.ndarray - Matriz 3D de tamaño N con valores 0 y 1.
    """
    # Inicializar la matriz 3D con ceros
    matriz_3d = np.zeros((N, N, N), dtype=int)

    # Leer el archivo y procesar cada línea
    with open(ruta_archivo, 'r') as archivo:
        for linea in archivo:
            # Separar las coordenadas (x, y, z)
            x, y, z, _ = map(int, linea.strip().split(','))

            # Asignar el valor 1 en la posición correspondiente
            if 0 <= x < N and 0 <= y < N and 0 <= z < N:
                matriz_3d[x, y, z] = 1

    return matriz_3d

In [6]:
def cargar_etiquetas_de_archivo(archivo, N):
    """
    Carga las etiquetas de un archivo de texto en una matriz 3D de tamaño N.

    """
    etiquetas = archivo.split('-')
    matriz_etiqueta = np.zeros((N,N,N))
    for i in range(3):
        etiqueta=etiquetas[i]
        if len(etiqueta)>0:
            etiqueta=etiqueta.split('_')
            etiqueta=list(map(int, etiqueta))
            dim_x=etiqueta[0]
            dim_y=etiqueta[1]
            dim_z=etiqueta[2]
            matriz_etiqueta[dim_x,dim_y,dim_z]=1
    return matriz_etiqueta

In [33]:
X = []
y = []

# Listar archivos en la carpeta
archivos = os.listdir(train_dir)

# Filtrar solo archivos (no directorios)
archivos = [archivo for archivo in archivos if os.path.isfile(os.path.join(train_dir, archivo))]
# Dimension de la matriz 3d
N = 30
# Recorrer cada archivo en la carpeta
for archivo in archivos:
    #print(f"Procesando archivo: {archivo}")
    y.append(cargar_etiquetas_de_archivo(archivo,N))
    X.append(cargar_matriz_3d_de_archivo(os.path.join(train_dir, archivo), N))
    #print(matriz_etiqueta)

# Convertir X y y a arrays de numpy
X = np.array(X)  # Shape (num_samples, 30, 30, 30)
y = np.array(y)  # Shape (num_samples, 30, 30, 30)

# Expandir la dimensión para el canal (necesario para Conv3D)
X = np.expand_dims(X, axis=-1)  # Ahora tiene forma (num_samples, 30, 30, 30, 1)
y = np.expand_dims(y, axis=-1)  # Ahora tiene forma (num_samples, 30, 30, 30, 1)

In [10]:
#print(X)

In [60]:
# Función de pérdidas basada en la distancia entre los puntos reales y predichos
def custom_loss(y_true, y_pred):
    """
    Función de pérdida personalizada para comparar nubes de puntos 3D,
    optimizada para puntos con valor 1.

    Args:
        y_true: Tensor de etiquetas verdaderas, con forma (batch_size, 30*30*30).
        y_pred: Tensor de predicciones, con forma (batch_size, 30*30*30).

    Returns:
        Pérdida calculada.
    """

    # Reshape para obtener coordenadas individuales (x, y, z)
    print('ok')
    y_true = tf.reshape(y_true, (-1, 30, 30, 30, 3))
    y_pred = tf.reshape(y_pred, (-1, 30, 30, 30, 3))
    print('ytrue')
    print(tf.shape(y_true))
    print('ypred')
    print(tf.shape(y_pred))

    # Máscara para seleccionar solo los puntos con valor 1 en el canal Z
    mask = tf.equal(y_true[..., 2], 1)
    print(mask)
    print('ok2')
    # Aplicar la máscara a las coordenadas
    points_true = tf.boolean_mask(y_true, mask)
    points_pred = tf.boolean_mask(y_pred, mask)
    print('ok3')
    # Ordenar los puntos según las coordenadas x, y y z
    def sort_points(points):
        # Calculate indices for sorting based on all dimensions
        indices = tf.argsort(points[:,0] + points[:,1] * 1000 + points[:,2] * 1000000)
        # Gather points based on sorted indices
        return tf.gather(points, indices, batch_dims=0)
    print('ok4')
    sorted_points_true = sort_points(points_true)
    sorted_points_pred = sort_points(points_pred)
    print('ok5')
    # Calcular la distancia euclidiana entre puntos correspondientes
    squared_difference = tf.square(sorted_points_true - sorted_points_pred)
    squared_distance = tf.reduce_sum(squared_difference, axis=-1)
    print('ok6')
    distance = tf.sqrt(squared_distance)
    print('ok7')
    # Calcular la pérdida por distancia (por ejemplo, la media)
    loss_distance = tf.reduce_mean(distance)
    print(loss_distance)
    return loss_distance



In [20]:
def distance_loss(y_true, y_pred):
    # Asumimos que y_true y y_pred son tensores de tamaño [batch_size, N, N, N]

    # Encontrar los índices de los puntos de valor 1 en y_true y y_pred
    true_indices = tf.where(tf.equal(y_true, 1))
    pred_indices = tf.where(tf.equal(y_pred, 1))

    # Definir la función de distancia euclidiana
    def euclidean_dist(a, b):
        return tf.sqrt(tf.reduce_sum(tf.square(a - b), axis=-1))

    # Dividir la matriz en 4 cuadrantes en el plano horizontal (primera y segunda dimensiones)
    def divide_quadrants(indices, shape):
        print(shape)
        half_x = shape[0] // 2
        half_y = shape[1] // 2
        quadrants = []
        quadrants.append(tf.boolean_mask(indices, (indices[:, 1] < half_x) & (indices[:, 2] < half_y))) # Cuadrante 1
        quadrants.append(tf.boolean_mask(indices, (indices[:, 1] < half_x) & (indices[:, 2] >= half_y))) # Cuadrante 2
        quadrants.append(tf.boolean_mask(indices, (indices[:, 1] >= half_x) & (indices[:, 2] < half_y))) # Cuadrante 3
        quadrants.append(tf.boolean_mask(indices, (indices[:, 1] >= half_x) & (indices[:, 2] >= half_y))) # Cuadrante 4
        return quadrants

    # Obtener las dimensiones de las matrices
    shape = tf.shape(y_true)

    # Dividir los índices de puntos en cuadrantes para y_true y y_pred
    true_quadrants = divide_quadrants(true_indices, shape)
    pred_quadrants = divide_quadrants(pred_indices, shape)

    total_loss = 0.0
    for i in range(4):  # Comparar los puntos en cada cuadrante
        if tf.size(true_quadrants[i]) > 0 and tf.size(pred_quadrants[i]) > 0:
            # Extraer los primeros puntos para comparar (suponemos que hay un solo punto en cada cuadrante)
            true_point = true_quadrants[i][0]
            pred_point = pred_quadrants[i][0]
            total_loss += euclidean_dist(true_point, pred_point)
        else:
            total_loss += 2 * N

    return total_loss

In [24]:
def distance_loss_improved(y_true, y_pred, weights=None):
    """
    Calcula la distancia promedio entre puntos 3D correspondientes en dos matrices,
    dividiendo el espacio en octantes.

    Args:
        y_true: Tensor de verdad terrestre con forma [batch_size, N, N, N].
        y_pred: Tensor de predicciones con forma [batch_size, N, N, N].
        weights: Tensor opcional de pesos para cada octante, con forma [8].

    Returns:
        Tensor escalar representando la pérdida total.
    """

     # Encontrar los índices de los puntos de valor 1 en y_true y y_pred
    true_indices = tf.where(tf.equal(y_true, 1))
    pred_indices = tf.where(tf.equal(y_pred, 1))

    # Definir la función de distancia euclidiana
    def euclidean_dist(a, b):
        return tf.sqrt(tf.reduce_sum(tf.square(a - b), axis=-1))

     # Dividir la matriz en 4 cuadrantes en el plano horizontal (primera y segunda dimensiones)
    def divide_quadrants(indices, shape):
        print(shape)
        half_x = shape[0] // 2
        half_y = shape[1] // 2
        quadrants = []
        quadrants.append(tf.boolean_mask(indices, (indices[:, 1] < half_x) & (indices[:, 2] < half_y))) # Cuadrante 1
        quadrants.append(tf.boolean_mask(indices, (indices[:, 1] < half_x) & (indices[:, 2] >= half_y))) # Cuadrante 2
        quadrants.append(tf.boolean_mask(indices, (indices[:, 1] >= half_x) & (indices[:, 2] < half_y))) # Cuadrante 3
        quadrants.append(tf.boolean_mask(indices, (indices[:, 1] >= half_x) & (indices[:, 2] >= half_y))) # Cuadrante 4
        return quadrants

    # Obtener las dimensiones de las matrices
    shape = tf.shape(y_true)[1:]  # Ignorar el batch_size

    # Dividir los índices de puntos en octantes para y_true y y_pred
    true_octants = divide_quadrants(true_indices, shape)
    pred_octants = divide_quadrants(pred_indices, shape)

    total_loss = 0.0
    for i in range(4):
        if tf.size(true_octants[i]) > 0 and tf.size(pred_octants[i]) > 0:
            # Calcular la distancia promedio entre todos los puntos del octante
            distances = tf.map_fn(lambda x: euclidean_dist(x[0], x[1]),
                                  (true_octants[i], pred_octants[i]),
                                  dtype=tf.float32)
            # Aplicar pesos si se proporcionan
            if weights is not None:
                distances *= weights[i]
            total_loss += tf.reduce_mean(distances)
        else:
            total_loss += 2 * N

    return total_loss

In [47]:
# MODELO v1


# Dividir en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir el modelo de red convolucional 3D
model = Sequential()

# Capa convolucional 3D 1
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(N,N,N, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Capa convolucional 3D 2
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Capa convolucional 3D 3
model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Aplanar las características para pasarlas a la capa densa
model.add(Flatten())

# Capa densa completamente conectada
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

# Capa de salida para predecir el mapa de etiquetas (y)
model.add(Dense(N*N*N, activation='sigmoid'))  # Redimensionar a una sola salida binaria por voxel

# Compilar el modelo
model.compile(optimizer='adam', loss=custom_loss2, metrics=['accuracy'])


In [48]:
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d_30 (Conv3D)                   │ (None, 28, 28, 28, 32)      │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_30 (MaxPooling3D)      │ (None, 14, 14, 14, 32)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_31 (Conv3D)                   │ (None, 12, 12, 12, 64)      │          55,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_31 (MaxPooling3D)      │ (None, 6, 6, 6, 64)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_32 (Conv3D)                   │ (None, 4, 4, 4, 128)        │         221,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling3d_32 (MaxPooling3D)      │ (None, 2, 2, 2, 128)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_10 (Flatten)                 │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 256)                 │         262,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 27000)               │       6,939,000 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,478,968 (28.53 MB)

 Trainable params: 7,478,968 (28.53 MB)

 Non-trainable params: 0 (0.00 B)

In [49]:
# Reshape de la salida y_train y y_test para que coincidan con la predicción
y_train_reshaped = y_train.reshape(y_train.shape[0], -1)  # Cambiar forma a (num_samples, 30*30*30)
y_test_reshaped = y_test.reshape(y_test.shape[0], -1)     # Cambiar forma a (num_samples, 30*30*30)

# Entrenar el modelo
model.fit(X_train, y_train_reshaped, epochs=5, batch_size=16, validation_data=(X_test, y_test_reshaped))

Epoch 1/5
pred
Tensor("compile_loss/custom_loss2/Equal_2:0", shape=(), dtype=bool)
true
Tensor("compile_loss/custom_loss2/strided_slice:0", shape=(), dtype=int32)


OperatorNotAllowedInGraphError: Iterating over a symbolic `tf.Tensor` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating this function with @tf.function. If that does not work, then you may be using an unsupported feature or your source code may not be visible to AutoGraph. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md#access-to-source-code for more information.

In [ ]:

model.save( BASE_FOLDER +'/modelo2.keras')  # The file needs to end with the .keras extension


In [ ]:
# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Pérdida: {loss}, Precisión: {accuracy}')